In [ ]:
%matplotlib inline
import tensorflow as tf
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:

dataset_path = "/kaggle/input/signlang/ArASL_Database_54K_Final/ArASL_Database_54K_Final"
label_path = "/kaggle/input/signlang/ArSL_Data_Labels.csv"

In [ ]:
image = glob(dataset_path + "/*/*.*")
len(image)

In [ ]:
# example of pixel normalization
from numpy import asarray
from PIL import Image
# load image
image = Image.open(image[0])
pixels = asarray(image)
# confirm pixel range is 0-255
print('Data Type: %s' % pixels.dtype)
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
# convert from integers to floats
pixels = pixels.astype('float32')
# normalize to the range 0-1
pixels /= 255.0
# confirm the normalization
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))

In [ ]:
df = pd.read_csv(label_path)
df.head()

In [ ]:
classes = df.Class.unique().tolist()
classes

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
training_generator = datagen.flow_from_directory(
        dataset_path,
        target_size=(64, 64),
        batch_size=32,
        color_mode="grayscale",
        classes = classes,
        subset='training')


validation_generator = datagen.flow_from_directory(
    dataset_path, # same directory as training data
    target_size=(64, 64),
    batch_size=32,
    color_mode="grayscale",
    classes = classes,
    subset='validation') # set as validation data

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow as tf
with tf.device('/device:GPU:0'):
    # build a 6-layer
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='softmax'))

model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, verbose=1)
checkpointer = tf.keras.callbacks.ModelCheckpoint('asl_char.h5',verbose=1,save_best_only=True)

optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
  data_generator,
  steps_per_epoch = data_generator.samples // 32,
  validation_data = validation_generator, 
  validation_steps = validation_generator.samples // 32,
  epochs=50, callbacks=[early_stopping, checkpointer])

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.experimental_new_converter = True
tflite_model = converter.convert()
open("asl_char.tflite", "wb").write(tflite_model)